# 📊 Book de Variáveis - Pagamento

## Contexto
Book com **120+ variáveis** para modelos de Risco, Inadimplência e Churn.

## Granularidade
- **Chave**: `SAFRA` (yyyyMM de `DAT_STATUS_FATURA`) + `NUM_CPF`

## Tabelas
| Tabela | Tipo |
|--------|------|
| `rawdata.pagamento` | Fato |
| `rawdata.forma_pagamento` | Dimensão |
| `rawdata.instituicao` | Dimensão |
| `rawdata.tipo_faturamento` | Dimensão |

In [1]:
# =============================================================================
# PARAMETROS DO NOTEBOOK (SAFRAS)
# =============================================================================
from datetime import date

LISTA_SAFRAS = ['202410', '202411', '202412', '202501', '202502', '202503']
MODO_ESCRITA = 'overwrite'

def safra_to_cutoff(safra):
    '''Converte safra (yyyyMM) para data de cutoff.'''
    return date(int(safra[:4]), int(safra[4:6]), 1).strftime('%Y-%m-%d')

print('Configuracao de Safras:')
for s in LISTA_SAFRAS:
    print(f'  Safra {s}: dados < {safra_to_cutoff(s)}')

StatementMeta(, 85f9e826-af27-47d4-aa9c-6fa004117507, 3, Finished, Available, Finished)

Configuracao de Safras:
  Safra 202503: dados < 2025-03-01


In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO AMBIENTE (importado do config centralizado)
# =============================================================================
import sys; sys.path.insert(0, "/lakehouse/default/Files/projeto-final")
from config.pipeline_config import SILVER_BASE, PATH_BOOK_PAGAMENTO

LAKEHOUSE_PATH = SILVER_BASE

# Paths de Entrada
PATH_PAGAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/pagamento"
PATH_FORMA_PAGAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/forma_pagamento"
PATH_INSTITUICAO = f"{LAKEHOUSE_PATH}/Tables/rawdata/instituicao"
PATH_TIPO_FATURAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/tipo_faturamento"

# Path de Saída
PATH_OUTPUT = PATH_BOOK_PAGAMENTO

print("✅ Configuração carregada")
print(f"📁 Output: {PATH_OUTPUT}")

In [3]:
# =============================================================================
# CARREGAMENTO DAS TABELAS DELTA
# =============================================================================

df_pagamento = spark.read.format("delta").load(PATH_PAGAMENTO)
print(f"📊 pagamento: {df_pagamento.count():,} registros")

df_forma_pagamento = spark.read.format("delta").load(PATH_FORMA_PAGAMENTO)
print(f"📊 forma_pagamento: {df_forma_pagamento.count():,} registros")

df_instituicao = spark.read.format("delta").load(PATH_INSTITUICAO)
print(f"📊 instituicao: {df_instituicao.count():,} registros")

df_tipo_faturamento = spark.read.format("delta").load(PATH_TIPO_FATURAMENTO)
print(f"📊 tipo_faturamento: {df_tipo_faturamento.count():,} registros")

print("\n✅ Todas as tabelas carregadas")

StatementMeta(, 85f9e826-af27-47d4-aa9c-6fa004117507, 5, Finished, Available, Finished)

📊 pagamento: 27,948,583 registros
📊 forma_pagamento: 9 registros
📊 instituicao: 550 registros
📊 tipo_faturamento: 72 registros

✅ Todas as tabelas carregadas


In [4]:
# =============================================================================
# REGISTRO DE VIEWS TEMPORÁRIAS
# =============================================================================

df_pagamento.createOrReplaceTempView("pagamento")
df_forma_pagamento.createOrReplaceTempView("dim_forma_pagamento")
df_instituicao.createOrReplaceTempView("dim_instituicao")
df_tipo_faturamento.createOrReplaceTempView("dim_tipo_faturamento")

print("✅ Views registradas: pagamento, dim_forma_pagamento, dim_instituicao, dim_tipo_faturamento")

StatementMeta(, 85f9e826-af27-47d4-aa9c-6fa004117507, 6, Finished, Available, Finished)

✅ Views registradas: pagamento, dim_forma_pagamento, dim_instituicao, dim_tipo_faturamento


In [ ]:
# =============================================================================
# QUERY SQL - BOOK DE VARIÁVEIS DE PAGAMENTO
# =============================================================================

SQL_TEMPLATE = """
WITH 
-- =============================================================================
-- BASE: Enriquecimento com Dimensões
-- =============================================================================
base_enrich AS (
    SELECT 
        p.*,
        CAST('{safra}' AS INT) AS SAFRA,
        
        -- Dimensão Forma Pagamento
        fp.DSC_FORMA_PAGAMENTO,
        
        -- Dimensão Instituição (via DW_BANCO)
        i.DSC_INSTITUICAO,
        i.DSC_TIPO_INSTITUICAO,
        
        -- Dimensão Tipo Faturamento
        tf.DSC_TIPO_FATURAMENTO,
        tf.DSC_TIPO_FATURAMENTO_ABREV,
        
        -- Flags Pivotadas: Status Pagamento (R/C/P/B)
        CASE WHEN p.IND_STATUS_PAGAMENTO = 'R' THEN 1 ELSE 0 END AS FLAG_STATUS_PAGAMENTO_R,
        CASE WHEN p.IND_STATUS_PAGAMENTO = 'C' THEN 1 ELSE 0 END AS FLAG_STATUS_PAGAMENTO_C,
        CASE WHEN p.IND_STATUS_PAGAMENTO = 'P' THEN 1 ELSE 0 END AS FLAG_STATUS_PAGAMENTO_P,
        CASE WHEN p.IND_STATUS_PAGAMENTO = 'B' THEN 1 ELSE 0 END AS FLAG_STATUS_PAGAMENTO_B,
        
        -- Flags Pivotadas: Status Fatura (C/O)
        CASE WHEN p.IND_STATUS_FATURA = 'C' THEN 1 ELSE 0 END AS FLAG_FATURA_FECHADA,
        CASE WHEN p.IND_STATUS_FATURA = 'O' THEN 1 ELSE 0 END AS FLAG_FATURA_ABERTA,
        
        -- Flags Pivotadas: Forma Pagamento (CA/PB/DD/PA)
        CASE WHEN p.COD_FORMA_PAGAMENTO = 'CA' THEN 1 ELSE 0 END AS FLAG_FORMA_CA,
        CASE WHEN p.COD_FORMA_PAGAMENTO = 'PB' THEN 1 ELSE 0 END AS FLAG_FORMA_PB,
        CASE WHEN p.COD_FORMA_PAGAMENTO = 'DD' THEN 1 ELSE 0 END AS FLAG_FORMA_DD,
        CASE WHEN p.COD_FORMA_PAGAMENTO = 'PA' THEN 1 ELSE 0 END AS FLAG_FORMA_PA,
        
        -- Flags Pivotadas: Tipo Pagamento (O/P/D/B)
        CASE WHEN p.COD_TIPO_PAGAMENTO = 'O' THEN 1 ELSE 0 END AS FLAG_TIPO_O,
        CASE WHEN p.COD_TIPO_PAGAMENTO = 'P' THEN 1 ELSE 0 END AS FLAG_TIPO_P,
        CASE WHEN p.COD_TIPO_PAGAMENTO = 'D' THEN 1 ELSE 0 END AS FLAG_TIPO_D,
        CASE WHEN p.COD_TIPO_PAGAMENTO = 'B' THEN 1 ELSE 0 END AS FLAG_TIPO_B,
        
        -- Flags Pivotadas: Alocação Crédito (PYM/CRT/CRTW)
        CASE WHEN p.COD_ALOCACAO_CREDITO = 'PYM' THEN 1 ELSE 0 END AS FLAG_ALOCACAO_PYM,
        CASE WHEN p.COD_ALOCACAO_CREDITO = 'CRT' THEN 1 ELSE 0 END AS FLAG_ALOCACAO_CRT,
        CASE WHEN p.COD_ALOCACAO_CREDITO = 'CRTW' THEN 1 ELSE 0 END AS FLAG_ALOCACAO_CRTW
        
    FROM pagamento p
    
    LEFT JOIN dim_forma_pagamento fp 
        ON p.DW_FORMA_PAGAMENTO = fp.DW_FORMA_PAGAMENTO
        
    LEFT JOIN dim_instituicao i 
        ON p.DW_BANCO = i.DW_INSTITUICAO
        
    LEFT JOIN dim_tipo_faturamento tf 
        ON p.DW_TIPO_FATURA = tf.DW_TIPO_FATURAMENTO
    WHERE p.DAT_STATUS_FATURA < '{data_cutoff}'
),

-- =============================================================================
-- AGREGAÇÃO PRINCIPAL: Por SAFRA + NUM_CPF
-- =============================================================================
agregado AS (
    SELECT 
        SAFRA,
        NUM_CPF,
        
        -- =====================================================================
        -- SEÇÃO 1: VOLUMETRIA
        -- =====================================================================
        COUNT(*) AS QTD_PAGAMENTOS_TOTAL,
        COUNT(DISTINCT CONTRATO) AS QTD_CONTRATOS,
        COUNT(DISTINCT SEQ_FATURA) AS QTD_FATURAS_DISTINTAS,
        COUNT(DISTINCT DW_FORMA_PAGAMENTO) AS QTD_FORMAS_PAGAMENTO,
        COUNT(DISTINCT DW_BANCO) AS QTD_BANCOS,
        COUNT(DISTINCT DW_TIPO_FATURA) AS QTD_TIPOS_FATURA,
        COUNT(DISTINCT DW_UN_NEGOCIO) AS QTD_UN_NEGOCIOS,
        COUNT(DISTINCT DW_AREA) AS QTD_AREAS,
        
        -- =====================================================================
        -- SEÇÃO 2: VALORES DE PAGAMENTO
        -- =====================================================================
        
        -- Pagamento Fatura
        SUM(COALESCE(VAL_PAGAMENTO_FATURA, 0)) AS VLR_PAGAMENTO_FATURA_TOTAL,
        AVG(COALESCE(VAL_PAGAMENTO_FATURA, 0)) AS VLR_PAGAMENTO_FATURA_MEDIO,
        MAX(COALESCE(VAL_PAGAMENTO_FATURA, 0)) AS VLR_PAGAMENTO_FATURA_MAX,
        STDDEV(COALESCE(VAL_PAGAMENTO_FATURA, 0)) AS VLR_PAGAMENTO_FATURA_STDDEV,
        
        -- Pagamento Item
        SUM(COALESCE(VAL_PAGAMENTO_ITEM, 0)) AS VLR_PAGAMENTO_ITEM_TOTAL,
        AVG(COALESCE(VAL_PAGAMENTO_ITEM, 0)) AS VLR_PAGAMENTO_ITEM_MEDIO,
        
        -- Pagamento Crédito
        SUM(COALESCE(VAL_PAGAMENTO_CREDITO, 0)) AS VLR_PAGAMENTO_CREDITO_TOTAL,
        AVG(COALESCE(VAL_PAGAMENTO_CREDITO, 0)) AS VLR_PAGAMENTO_CREDITO_MEDIO,
        MAX(COALESCE(VAL_PAGAMENTO_CREDITO, 0)) AS VLR_PAGAMENTO_CREDITO_MAX,
        
        -- Valor Atual e Original
        SUM(COALESCE(VAL_ATUAL_PAGAMENTO, 0)) AS VLR_ATUAL_PAGAMENTO_TOTAL,
        AVG(COALESCE(VAL_ATUAL_PAGAMENTO, 0)) AS VLR_ATUAL_PAGAMENTO_MEDIO,
        SUM(COALESCE(VAL_ORIGINAL_PAGAMENTO, 0)) AS VLR_ORIGINAL_PAGAMENTO_TOTAL,
        AVG(COALESCE(VAL_ORIGINAL_PAGAMENTO, 0)) AS VLR_ORIGINAL_PAGAMENTO_MEDIO,
        
        -- Baixa Atividade
        SUM(COALESCE(VAL_BAIXA_ATIVIDADE, 0)) AS VLR_BAIXA_ATIVIDADE_TOTAL,
        AVG(COALESCE(VAL_BAIXA_ATIVIDADE, 0)) AS VLR_BAIXA_ATIVIDADE_MEDIO,
        
        -- Juros e Multas
        SUM(COALESCE(VAL_JUROS_MULTAS_ITEM, 0)) AS VLR_JUROS_MULTAS_TOTAL,
        AVG(COALESCE(VAL_JUROS_MULTAS_ITEM, 0)) AS VLR_JUROS_MULTAS_MEDIO,
        MAX(COALESCE(VAL_JUROS_MULTAS_ITEM, 0)) AS VLR_JUROS_MULTAS_MAX,
        COUNT(CASE WHEN VAL_JUROS_MULTAS_ITEM > 0 THEN 1 END) AS QTD_PAGAMENTOS_COM_JUROS,
        
        -- Multa Equipamento
        SUM(COALESCE(VAL_MULTA_EQUIP_ITEM, 0)) AS VLR_MULTA_EQUIP_TOTAL,
        COUNT(CASE WHEN VAL_MULTA_EQUIP_ITEM > 0 THEN 1 END) AS QTD_MULTAS_EQUIP,
        
        -- =====================================================================
        -- SEÇÃO 3: PIVOT POR STATUS PAGAMENTO (R/C/P/B)
        -- =====================================================================
        SUM(FLAG_STATUS_PAGAMENTO_R) AS QTD_STATUS_R,
        SUM(FLAG_STATUS_PAGAMENTO_C) AS QTD_STATUS_C,
        SUM(FLAG_STATUS_PAGAMENTO_P) AS QTD_STATUS_P,
        SUM(FLAG_STATUS_PAGAMENTO_B) AS QTD_STATUS_B,
        
        SUM(CASE WHEN FLAG_STATUS_PAGAMENTO_R = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_STATUS_R,
        SUM(CASE WHEN FLAG_STATUS_PAGAMENTO_C = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_STATUS_C,
        SUM(CASE WHEN FLAG_STATUS_PAGAMENTO_P = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_STATUS_P,
        SUM(CASE WHEN FLAG_STATUS_PAGAMENTO_B = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_STATUS_B,
        
        -- =====================================================================
        -- SEÇÃO 4: PIVOT POR STATUS FATURA (C/O)
        -- =====================================================================
        SUM(FLAG_FATURA_FECHADA) AS QTD_FATURA_FECHADA,
        SUM(FLAG_FATURA_ABERTA) AS QTD_FATURA_ABERTA,
        SUM(CASE WHEN FLAG_FATURA_FECHADA = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FATURA_FECHADA,
        SUM(CASE WHEN FLAG_FATURA_ABERTA = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FATURA_ABERTA,
        
        -- =====================================================================
        -- SEÇÃO 5: PIVOT POR FORMA PAGAMENTO (CA/PB/DD/PA)
        -- =====================================================================
        SUM(FLAG_FORMA_CA) AS QTD_FORMA_CA,
        SUM(FLAG_FORMA_PB) AS QTD_FORMA_PB,
        SUM(FLAG_FORMA_DD) AS QTD_FORMA_DD,
        SUM(FLAG_FORMA_PA) AS QTD_FORMA_PA,
        
        SUM(CASE WHEN FLAG_FORMA_CA = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FORMA_CA,
        SUM(CASE WHEN FLAG_FORMA_PB = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FORMA_PB,
        SUM(CASE WHEN FLAG_FORMA_DD = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FORMA_DD,
        SUM(CASE WHEN FLAG_FORMA_PA = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_FORMA_PA,
        
        -- =====================================================================
        -- SEÇÃO 6: PIVOT POR TIPO PAGAMENTO (O/P/D/B)
        -- =====================================================================
        SUM(FLAG_TIPO_O) AS QTD_TIPO_O,
        SUM(FLAG_TIPO_P) AS QTD_TIPO_P,
        SUM(FLAG_TIPO_D) AS QTD_TIPO_D,
        SUM(FLAG_TIPO_B) AS QTD_TIPO_B,
        
        SUM(CASE WHEN FLAG_TIPO_O = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_TIPO_O,
        SUM(CASE WHEN FLAG_TIPO_P = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_TIPO_P,
        SUM(CASE WHEN FLAG_TIPO_D = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_TIPO_D,
        SUM(CASE WHEN FLAG_TIPO_B = 1 THEN COALESCE(VAL_PAGAMENTO_FATURA, 0) ELSE 0 END) AS VLR_TIPO_B,
        
        -- =====================================================================
        -- SEÇÃO 7: PIVOT POR ALOCAÇÃO CRÉDITO (PYM/CRT/CRTW)
        -- =====================================================================
        SUM(FLAG_ALOCACAO_PYM) AS QTD_ALOCACAO_PYM,
        SUM(FLAG_ALOCACAO_CRT) AS QTD_ALOCACAO_CRT,
        SUM(FLAG_ALOCACAO_CRTW) AS QTD_ALOCACAO_CRTW,
        
        SUM(CASE WHEN FLAG_ALOCACAO_PYM = 1 THEN COALESCE(VAL_PAGAMENTO_CREDITO, 0) ELSE 0 END) AS VLR_ALOCACAO_PYM,
        SUM(CASE WHEN FLAG_ALOCACAO_CRT = 1 THEN COALESCE(VAL_PAGAMENTO_CREDITO, 0) ELSE 0 END) AS VLR_ALOCACAO_CRT,
        
        -- =====================================================================
        -- SEÇÃO 8: MÉTRICAS TEMPORAIS
        -- =====================================================================
        MIN(DAT_STATUS_FATURA) AS DT_PRIMEIRA_FATURA,
        MAX(DAT_STATUS_FATURA) AS DT_ULTIMA_FATURA,
        MIN(DAT_CRIACAO_DW) AS DT_PRIMEIRA_CRIACAO_DW,
        MAX(DAT_CRIACAO_DW) AS DT_ULTIMA_CRIACAO_DW,
        
        DATEDIFF(MAX(DAT_STATUS_FATURA), MIN(DAT_STATUS_FATURA)) AS DIAS_ENTRE_FATURAS,
        DATEDIFF(TO_DATE('{data_cutoff}'), MAX(DAT_STATUS_FATURA)) AS DIAS_DESDE_ULTIMA_FATURA,
        
        AVG(DATEDIFF(DAT_BAIXA_ATIVIDADE, DAT_CRIACAO_ATIVIDADE)) AS DIAS_MEDIO_BAIXA,
        AVG(DATEDIFF(DAT_DEPOSITO_ATIVIDADE, DAT_CRIACAO_ATIVIDADE)) AS DIAS_MEDIO_DEPOSITO,
        
        -- Meses distintos com atividade
        COUNT(DISTINCT DATE_FORMAT(DAT_STATUS_FATURA, 'yyyyMM')) AS QTD_MESES_ATIVOS,
        
        -- =====================================================================
        -- SEÇÃO 9: DADOS DAS DIMENSÕES
        -- =====================================================================
        FIRST(DSC_FORMA_PAGAMENTO) AS FORMA_PAGAMENTO_PRINCIPAL,
        FIRST(DSC_TIPO_INSTITUICAO) AS TIPO_INSTITUICAO_PRINCIPAL,
        FIRST(DSC_TIPO_FATURAMENTO_ABREV) AS TIPO_FATURAMENTO_PRINCIPAL,
        COUNT(DISTINCT DSC_TIPO_INSTITUICAO) AS QTD_TIPOS_INSTITUICAO
        
    FROM base_enrich
    GROUP BY SAFRA, NUM_CPF
)

-- =============================================================================
-- RESULTADO FINAL COM TAXAS, SCORES E SEGMENTAÇÃO
-- =============================================================================
SELECT 
    a.*,
    
    -- =========================================================================
    -- SEÇÃO 10: TAXAS E PROPORÇÕES
    -- =========================================================================
    
    -- Taxa por Status Pagamento
    a.QTD_STATUS_R / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_R,
    a.QTD_STATUS_C / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_C,
    a.QTD_STATUS_P / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_P,
    a.QTD_STATUS_B / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_STATUS_B,
    
    -- Taxa por Forma Pagamento
    a.QTD_FORMA_CA / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_FORMA_CA,
    a.QTD_FORMA_PB / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_FORMA_PB,
    a.QTD_FORMA_DD / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_FORMA_DD,
    a.QTD_FORMA_PA / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_FORMA_PA,
    
    -- Taxa Fatura Aberta
    a.QTD_FATURA_ABERTA / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_FATURA_ABERTA,
    
    -- Taxa de Juros/Multas
    a.QTD_PAGAMENTOS_COM_JUROS / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0) AS TAXA_PAGAMENTOS_COM_JUROS,
    a.VLR_JUROS_MULTAS_TOTAL / NULLIF(a.VLR_PAGAMENTO_FATURA_TOTAL, 0) AS TAXA_JUROS_SOBRE_PAGAMENTO,
    
    -- =========================================================================
    -- SEÇÃO 11: ÍNDICES DE ESTABILIDADE
    -- =========================================================================
    
    -- Coeficiente de Variação
    a.VLR_PAGAMENTO_FATURA_STDDEV / NULLIF(a.VLR_PAGAMENTO_FATURA_MEDIO, 0) AS COEF_VARIACAO_PAGAMENTO,
    
    -- Índice de Concentração
    a.VLR_PAGAMENTO_FATURA_MAX / NULLIF(a.VLR_PAGAMENTO_FATURA_TOTAL, 0) AS INDICE_CONCENTRACAO,
    
    -- Ticket Médio por Contrato
    a.VLR_PAGAMENTO_FATURA_TOTAL / NULLIF(a.QTD_CONTRATOS, 0) AS VLR_TICKET_MEDIO_CONTRATO,
    
    -- Diferença Original vs Atual
    a.VLR_ORIGINAL_PAGAMENTO_TOTAL - a.VLR_ATUAL_PAGAMENTO_TOTAL AS VLR_DIFERENCA_ORIG_ATUAL,
    
    -- =========================================================================
    -- SEÇÃO 12: SCORE DE RISCO (0-100)
    -- =========================================================================
    LEAST(100, GREATEST(0,
        -- Componente Faturas Abertas (peso 25)
        COALESCE(a.QTD_FATURA_ABERTA / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0), 0) * 25 +
        -- Componente Juros/Multas (peso 25)
        LEAST(1, COALESCE(a.VLR_JUROS_MULTAS_TOTAL / NULLIF(a.VLR_PAGAMENTO_FATURA_TOTAL, 0), 0) * 5) * 25 +
        -- Componente Status B (peso 20)
        COALESCE(a.QTD_STATUS_B / NULLIF(CAST(a.QTD_PAGAMENTOS_TOTAL AS DOUBLE), 0), 0) * 20 +
        -- Componente Volatilidade (peso 15)
        LEAST(1, COALESCE(a.VLR_PAGAMENTO_FATURA_STDDEV / NULLIF(a.VLR_PAGAMENTO_FATURA_MEDIO, 0), 0)) * 15 +
        -- Componente Multas Equip (peso 15)
        CASE WHEN a.QTD_MULTAS_EQUIP > 0 THEN 15 ELSE 0 END
    )) AS SCORE_RISCO,
    
    -- FLAGS MANDATÓRIAS
    CASE 
        WHEN a.QTD_FATURA_ABERTA > a.QTD_FATURA_FECHADA THEN 1
        WHEN a.VLR_JUROS_MULTAS_TOTAL > a.VLR_PAGAMENTO_FATURA_TOTAL * 0.1 THEN 1
        WHEN a.QTD_STATUS_B > a.QTD_PAGAMENTOS_TOTAL * 0.3 THEN 1
        ELSE 0
    END AS FLAG_ALTO_RISCO,
    
    CASE 
        WHEN a.QTD_FATURA_ABERTA = 0 
            AND a.VLR_JUROS_MULTAS_TOTAL = 0
            AND a.QTD_STATUS_C > a.QTD_PAGAMENTOS_TOTAL * 0.9
        THEN 1
        ELSE 0
    END AS FLAG_BAIXO_RISCO,
    
    -- SEGMENTO DE RISCO
    CASE 
        WHEN a.QTD_FATURA_ABERTA > a.QTD_FATURA_FECHADA 
            OR a.VLR_JUROS_MULTAS_TOTAL > a.VLR_PAGAMENTO_FATURA_TOTAL * 0.2 THEN 'CRITICO'
        WHEN a.QTD_FATURA_ABERTA > 0 
            OR a.VLR_JUROS_MULTAS_TOTAL > a.VLR_PAGAMENTO_FATURA_TOTAL * 0.1 THEN 'ALTO'
        WHEN a.VLR_JUROS_MULTAS_TOTAL > 0 
            OR a.QTD_STATUS_P > a.QTD_PAGAMENTOS_TOTAL * 0.1 THEN 'MEDIO'
        ELSE 'BAIXO'
    END AS SEGMENTO_RISCO,
    
    -- DATA DE PROCESSAMENTO
    CURRENT_TIMESTAMP() AS DT_PROCESSAMENTO
    
FROM agregado a
"""

print("✅ Query SQL definida")
print("📊 Estimativa: 120+ variáveis")

In [6]:
# =============================================================================
# LOOP PRINCIPAL - PROCESSAR CADA SAFRA
# =============================================================================
resultados = []

for i, safra in enumerate(LISTA_SAFRAS):
    print(f'\n[{i+1}/{len(LISTA_SAFRAS)}] Processando SAFRA {safra}...')
    data_cutoff = safra_to_cutoff(safra)
    print(f'  Cutoff: {data_cutoff}')
    
    query = SQL_TEMPLATE.format(safra=safra, data_cutoff=data_cutoff)
    
    df_safra = spark.sql(query)
    df_safra.cache()
    cnt = df_safra.count()
    print(f'  Registros: {cnt:,}')
    resultados.append({'safra': safra, 'cnt': cnt})
    
    modo = 'overwrite' if i == 0 and MODO_ESCRITA == 'overwrite' else 'append'
    df_safra.write.format('delta').mode(modo).partitionBy('SAFRA').option('overwriteSchema', 'true' if modo == 'overwrite' else 'false').save(PATH_OUTPUT)
    print(f'  Salvo (modo: {modo})')
    df_safra.unpersist()

print('\n' + '='*50)
print('CONCLUIDO!')
total = sum(r['cnt'] for r in resultados)
print(f'Total: {total:,} registros em {len(LISTA_SAFRAS)} safras')

StatementMeta(, 85f9e826-af27-47d4-aa9c-6fa004117507, 8, Submitted, Running, Running)


[1/1] Processando SAFRA 202503...
  Cutoff: 2025-03-01


---
# 📖 Dicionário de Variáveis

| Categoria | Variáveis | Descrição |
|-----------|-----------|----------|
| Volumetria | QTD_PAGAMENTOS_TOTAL, QTD_CONTRATOS, etc | Contagens |
| Valores | VLR_PAGAMENTO_*, VLR_JUROS_* | Métricas financeiras |
| Status (R/C/P/B) | QTD_STATUS_*, VLR_STATUS_*, TAXA_STATUS_* | Pivot por status |
| Forma (CA/PB/DD/PA) | QTD_FORMA_*, VLR_FORMA_*, TAXA_FORMA_* | Pivot por forma pagto |
| Temporal | DT_*, DIAS_* | Métricas de tempo |
| Índices | COEF_VARIACAO, INDICE_CONCENTRACAO | Estabilidade |
| Scores | SCORE_RISCO (0-100), FLAG_ALTO/BAIXO_RISCO, SEGMENTO_RISCO | Classificação |